In [1]:
import utide
import requests
from dateutil import parser
import numpy as np
import pytz
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

def get_coops_data(station, start_date, end_date, product='hourly_height', units='metric'):
    """
    units can be 'english' or 'metric'

    product options include 'water_level', 'hourly_height', 'predictions'
    from https://tidesandcurrents.noaa.gov/api/
    Option	Description
    water_level	Preliminary or verified water levels, depending on availability.
    air_temperature	Air temperature as measured at the station.
    water_temperature	Water temperature as measured at the station.
    wind	Wind speed, direction, and gusts as measured at the station.
    air_pressure	Barometric pressure as measured at the station.
    air_gap	Air Gap (distance between a bridge and the water's surface) at the station.
    conductivity	The water's conductivity as measured at the station.
    visibility	Visibility from the station's visibility sensor. A measure of atmospheric clarity.
    humidity	Relative humidity as measured at the station.
    salinity	Salinity and specific gravity data for the station.
    hourly_height	Verified hourly height water level data for the station.
    high_low	Verified high/low water level data for the station.
    daily_mean	Verified daily mean water level data for the station.
    monthly_mean	Verified monthly mean water level data for the station.
    one_minute_water_level	One minute water level data for the station.
    predictions	6 minute predictions water level data for the station.
    datums	datums data for the stations.
    currents	Currents data for currents stations.
    """

    url = 'http://tidesandcurrents.noaa.gov/api/datagetter?product=' \
    + product \
    + '&application=NOS.COOPS.TAC.WL&begin_date=' \
    + str(start_date) \
    + '&end_date=' \
    + str(end_date) \
    + '&datum=MLLW&station=' \
    + str(station) \
    + '&time_zone=GMT&units=' \
    + units \
    + '&format=json'

    payload = requests.get(url).json()

    if 'error' in payload.keys():
        raise ValueError('Error in returning dataset. Time requested too long?')

    t = []
    v = []
    if product == 'water_level' or product == 'hourly_height':
        d = payload['data']
    elif product == 'predictions':
        d = payload['predictions']
        
    for n in range(len(d)):
        t.append(pytz.utc.localize(parser.parse(d[n]['t'])))
        try:
            v.append(float(d[n]['v']))
        except:
            v.append(np.nan)
    t = np.array(t)
    v = np.array(v)

    n = {}
    n['time'] = t
    n['v'] = v

    return n

In [2]:
harm = pd.read_csv('data/cherry_point_harmonics.txt', sep='\t')
dfyear = pd.DataFrame(get_coops_data('9449424', '20180101', '20181231'))
timeyear = mdates.date2num(dfyear.time)
coefyear = utide.solve(timeyear, dfyear['v'].values,
                       lat=48.9,
                       method='ols',
                       conf_int='MC')
tideyear = utide.reconstruct(timeyear, coefyear)

solve: matrix prep ... solution ... diagnostics ... done.
prep/calcs ... done.


In [3]:
coefs = []
times = []
tides = []
for end in pd.date_range('2018-01-01', '2018-12-31', freq='M'):
    df = pd.DataFrame(get_coops_data('9449424', '20180101', end.strftime('%Y%m%d')))
    time = mdates.date2num(df.time)
    coef = utide.solve(time, df['v'].values,
                       lat=48.9,
                       method='ols',
                       conf_int='MC')
    coefs.append(coef)
    times.append(time)
    tides.append(utide.reconstruct(time, coef))

solve: matrix prep ... solution ... diagnostics ... done.
prep/calcs ... done.
solve: matrix prep ... solution ... diagnostics ... done.
prep/calcs ... done.
solve: matrix prep ... solution ... diagnostics ... done.
prep/calcs ... done.
solve: matrix prep ... solution ... diagnostics ... done.
prep/calcs ... done.
solve: matrix prep ... solution ... diagnostics ... done.
prep/calcs ... done.
solve: matrix prep ... solution ... diagnostics ... done.
prep/calcs ... done.
solve: matrix prep ... solution ... diagnostics ... done.
prep/calcs ... done.
solve: matrix prep ... solution ... diagnostics ... done.
prep/calcs ... done.
solve: matrix prep ... solution ... diagnostics ... done.
prep/calcs ... done.
solve: matrix prep ... solution ... diagnostics ... done.
prep/calcs ... done.
solve: matrix prep ... solution ... diagnostics ... done.
prep/calcs ... done.
solve: matrix prep ... solution ... diagnostics ... done.
prep/calcs ... done.


In [4]:
subtides = []
for n in range(len(coefs)):
    subtides.append(utide.reconstruct(timeyear, coefs[n]))

prep/calcs ... done.
prep/calcs ... done.
prep/calcs ... done.
prep/calcs ... done.
prep/calcs ... done.
prep/calcs ... done.
prep/calcs ... done.
prep/calcs ... done.
prep/calcs ... done.
prep/calcs ... done.
prep/calcs ... done.
prep/calcs ... done.


# Use the slider to see how the tidal predictions vary when using different amounts of data to make the prediction

In [5]:
amp = harm['Amplitude'][harm['Name'] == 'K1'].values
pcterr = []

for coef in coefs:
    if 'K1' in coef.name:
        pcterr.append(100*(coef.A[coef.name == 'K1']-amp)/amp)
    else:
        pcterr.append(np.nan)
        
def plot_func(months):
    n = months - 1

    plt.figure(figsize=(10,8))
    plt.subplot(2,1,1)
    plt.plot(dfyear.time, dfyear['v'].values, label='Measured', lw=0.75)
    plt.plot(dfyear.time, subtides[n].h, label='Predicted', lw=0.75)
    plt.ylim(-3, 7)
    plt.title(f'Using {months} month to make tidal prediction')
    plt.ylabel('Water level [m]')
    plt.xlabel('Date')
    plt.legend(loc='upper left')
    
    plt.subplot(2,1,2)
    plt.plot(np.arange(len(coefs))+1, pcterr,'o',)
    plt.plot(months, pcterr[n], 'o', ms=10)
    plt.axhline(0, ls='--', c='grey')
    plt.ylabel('Percent error in K1 constituent')
    plt.xlabel('Number of months included in tidal prediction')
    plt.xticks(np.linspace(1,12,12))

interactive_plot = interactive(plot_func, months = widgets.IntSlider(value=1,
                                               min=1,
                                               max=12,
                                               step=1))
output = interactive_plot.children[-1]
output.layout.height = '500px'
interactive_plot

interactive(children=(IntSlider(value=1, description='months', max=12, min=1), Output(layout=Layout(height='50…